<a href="https://colab.research.google.com/github/hotekagi/me_practice/blob/main/%E8%BC%AA%E8%AC%9B(%E8%A1%8C%E5%88%97%E3%81%AE%E3%82%B9%E3%82%B1%E3%83%83%E3%83%81).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**GPU使えば大きな行列でも数値実験できるので実装してみました**

セットアップ

In [8]:
import torch

def make_S(n: int, r: int):
    # make a sparse embedding matrix S (p.18,19)
    rows = torch.randint(r, size=(n,)).tolist()
    cols = torch.arange(n).tolist()
    data = torch.where(torch.rand(n) < 0.5, torch.ones(n), -torch.ones(n))

    S = torch.sparse_coo_tensor(
        indices=[rows, cols], values=data, size=(r, n), dtype=torch.float64
    )
    return S

def make_A(n: int, d: int, dense=0.3, max=100):
    offset = max * (1-dense)
    sparse_matrix = torch.randint(max, (n,d))
    sparse_matrix[sparse_matrix < offset] = offset
    return sparse_matrix - offset

def colorizeOutput(text, color):
    return f"\033[3{color}m{text}\033[00m"

CPU版

In [9]:
eps = 0.01
n = 10000
d = 10
r = int(d * d /eps)

A = make_A(n, d).to(torch.float32)
original_coeff = torch.randint(100, (d,)).to(torch.float32)
print("original_coeff", original_coeff)
b = torch.mv(A, original_coeff)  + torch.normal(0, 1, size=(n,)).to(torch.float32)

A_pinv = torch.linalg.pinv(A)
actual_coeff = A_pinv @ b

S = make_S(n, r).to(torch.float32)
SA = torch.mm(S, A)
Sb = torch.mv(S, b)

SA_pinv = torch.linalg.pinv(SA)
estimated_coeff = SA_pinv @ Sb

criteria = torch.nn.MSELoss()
original_SE = criteria(A@original_coeff, b) # np.linalg.norm(A @ original_coeff - b, ord=2)
actual_SE = criteria(A@actual_coeff, b)
estimated_SE = criteria(A@estimated_coeff, b)

print(colorizeOutput(f" Original Coeff: {original_coeff}", 6))
print(colorizeOutput(f"   Actual Coeff: {actual_coeff}", 4))
print(colorizeOutput(f"Estimated Coeff: {estimated_coeff}", 2))

print("=" * 10)

print(colorizeOutput(f" Original Mean Squared Error: {original_SE}", 6))
print(colorizeOutput(f"   Actual Mean Squared Error: {actual_SE}", 4))
print(colorizeOutput(f"Estimated Mean Squared Error: {estimated_SE}", 2))

print("eps:", eps)
print("error rate:", (float(estimated_SE) / float(actual_SE))-1)

original_coeff tensor([57., 66.,  3., 88., 75., 70., 57., 45., 11.,  9.])
 Original Coeff: tensor([57., 66.,  3., 88., 75., 70., 57., 45., 11.,  9.])
   Actual Coeff: tensor([57.0020, 65.9994,  2.9989, 87.9977, 74.9998, 69.9996, 57.0007, 44.9979,
        11.0014,  8.9989])
Estimated Coeff: tensor([57.0019, 65.9977,  3.0004, 87.9969, 75.0012, 70.0004, 57.0015, 44.9958,
        11.0014,  8.9986])
 Original Mean Squared Error: 0.9834068417549133
   Actual Mean Squared Error: 0.9818774461746216
Estimated Mean Squared Error: 0.9827638864517212
eps: 0.01
error rate: 0.0009028013430323334


GPU版

In [14]:
!nvidia-smi

Wed May 10 08:36:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    25W /  70W |    209MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
eps = 0.01
n = 100000
d = 100
r = int(d * d /eps)

A = make_A(n, d).to('cuda').to(torch.float32)
original_coeff = torch.randint(100, (d,)).to('cuda').to(torch.float32)
print("original_coeff", original_coeff)
b = torch.mv(A, original_coeff)  + torch.normal(0, 1, size=(n,)).to('cuda').to(torch.float32)

A_pinv = torch.linalg.pinv(A)
actual_coeff = A_pinv @ b

S = make_S(n, r).to('cuda').to(torch.float32)
SA = torch.mm(S, A)
Sb = torch.mv(S, b)

SA_pinv = torch.linalg.pinv(SA)
estimated_coeff = SA_pinv @ Sb

criteria = torch.nn.MSELoss()
original_SE = criteria(A@original_coeff, b) # np.linalg.norm(A @ original_coeff - b, ord=2)
actual_SE = criteria(A@actual_coeff, b)
estimated_SE = criteria(A@estimated_coeff, b)

print(colorizeOutput(f" Original Coeff: {original_coeff}", 6))
print(colorizeOutput(f"   Actual Coeff: {actual_coeff}", 4))
print(colorizeOutput(f"Estimated Coeff: {estimated_coeff}", 2))

print("=" * 10)

print(colorizeOutput(f" Original Mean Squared Error: {original_SE}", 6))
print(colorizeOutput(f"   Actual Mean Squared Error: {actual_SE}", 4))
print(colorizeOutput(f"Estimated Mean Squared Error: {estimated_SE}", 2))

print("eps:", eps)
print("error rate:", (float(estimated_SE) / float(actual_SE))-1)

original_coeff tensor([16., 38., 39., 23.,  7.,  0., 17., 18., 81., 86., 12., 93.,  5., 27.,
        52., 49., 92., 17., 94., 15., 74., 42., 40., 38., 91., 99., 57., 29.,
        91., 73., 31., 94., 24., 64., 50.,  7., 18., 77., 14., 76., 59., 62.,
        19., 52., 78., 33., 78., 18.,  9., 71., 27.,  7., 73., 64., 42.,  6.,
        42., 86., 83., 13., 81., 21., 55., 93., 30., 73., 83., 10., 40., 53.,
        79., 70.,  1., 63., 71., 92., 21., 18., 92., 46., 55., 12., 92., 79.,
        75., 49., 33., 95., 43., 13., 99., 54., 49., 24., 91., 73.,  5., 23.,
        78., 85.], device='cuda:0')
 Original Coeff: tensor([16., 38., 39., 23.,  7.,  0., 17., 18., 81., 86., 12., 93.,  5., 27.,
        52., 49., 92., 17., 94., 15., 74., 42., 40., 38., 91., 99., 57., 29.,
        91., 73., 31., 94., 24., 64., 50.,  7., 18., 77., 14., 76., 59., 62.,
        19., 52., 78., 33., 78., 18.,  9., 71., 27.,  7., 73., 64., 42.,  6.,
        42., 86., 83., 13., 81., 21., 55., 93., 30., 73., 83., 10., 40., 5

In [11]:
for eps in [0.1, 0.01, 0.001]:
    n = 10000
    d = 10
    r = int(d * d /eps)

    A = make_A(n, d).to('cuda').to(torch.float32)
    original_coeff = torch.randint(100, (d,)).to('cuda').to(torch.float32)
    print("original_coeff", original_coeff)
    b = torch.mv(A, original_coeff)  + torch.normal(0, 1, size=(n,)).to('cuda').to(torch.float32)

    A_pinv = torch.linalg.pinv(A)
    actual_coeff = A_pinv @ b

    S = make_S(n, r).to('cuda').to(torch.float32)
    SA = torch.mm(S, A)
    Sb = torch.mv(S, b)

    SA_pinv = torch.linalg.pinv(SA)
    estimated_coeff = SA_pinv @ Sb

    criteria = torch.nn.MSELoss()
    original_SE = criteria(A@original_coeff, b) # np.linalg.norm(A @ original_coeff - b, ord=2)
    actual_SE = criteria(A@actual_coeff, b)
    estimated_SE = criteria(A@estimated_coeff, b)

    # print(colorizeOutput(f" Original Coeff: {original_coeff}", 6))
    # print(colorizeOutput(f"   Actual Coeff: {actual_coeff}", 4))
    # print(colorizeOutput(f"Estimated Coeff: {estimated_coeff}", 2))

    print("=" * 10)

    print(colorizeOutput(f" Original Mean Squared Error: {original_SE}", 6))
    print(colorizeOutput(f"   Actual Mean Squared Error: {actual_SE}", 4))
    print(colorizeOutput(f"Estimated Mean Squared Error: {estimated_SE}", 2))

    print("eps:", eps)
    print("error rate:", (float(estimated_SE) / float(actual_SE))-1)

original_coeff tensor([28., 25., 65., 39., 72., 27., 77., 12.,  8., 38.], device='cuda:0')
 Original Mean Squared Error: 0.9891179203987122
   Actual Mean Squared Error: 0.9877382516860962
Estimated Mean Squared Error: 0.9949027299880981
eps: 0.1
error rate: 0.007253417886542213
original_coeff tensor([42., 32., 69., 82.,  5.,  8., 36., 20., 32., 57.], device='cuda:0')
 Original Mean Squared Error: 0.9881947040557861
   Actual Mean Squared Error: 0.9872143268585205
Estimated Mean Squared Error: 0.987934947013855
eps: 0.01
error rate: 0.0007299530970419355
original_coeff tensor([58., 86., 96., 70., 25., 15., 49., 74., 34., 92.], device='cuda:0')
 Original Mean Squared Error: 1.0216723680496216
   Actual Mean Squared Error: 1.021004319190979
Estimated Mean Squared Error: 1.0210603475570679
eps: 0.001
error rate: 5.4875738560333787e-05
